In [ ]:
!pip install catboost

In [114]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

In [115]:
train_ = pd.read_csv('Train.csv')
test_ = pd.read_csv('Test.csv')
submission_ = pd.read_csv('SampleSubmission.csv')

In [116]:
from sklearn.model_selection import KFold
fold_train = []
kf = KFold(n_splits=5, shuffle=False)
for r, (train_index, test_index) in enumerate(kf.split(train_)):
  test = train_.iloc[test_index]

  X_test = []
  X_test_columns = test.columns
  for v in test.values:
    info = v[:8]
    binary = v[8:]
    index = [k for k, i in enumerate(binary) if i == 1]
    for i in index:
      for k in range(len(binary)):
        if k == i:
          binary_transformed = list(copy.copy(binary))
          binary_transformed[i] = 0
          X_test.append(list(info) + binary_transformed + [8+k])

  X_test = pd.DataFrame(X_test)
  X_test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
          'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
          '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
          'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_num']

  X_test['ID'] = [str(r)+'_'+str(i) for i in range(X_test.shape[0])]

  test = []
  test_columns = X_test.columns

  for v in X_test.values:
    info = v[:8]
    binary = v[8:-1]
    num = v[-1]
    index = [k for k, i in enumerate(binary) if i == 1]
    for k in range(len(binary)):
        if k not in index:
          if 8+k == num:
            test.append(list(info) + list(binary) + [test_columns[8+k]] + [1])
          else:
            test.append(list(info) + list(binary) + [test_columns[8+k]] + [0])
  test = pd.DataFrame(test)
  test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred', 'target']

  test = test[['ID', 'product_pred', 'target']]
  test['ID X PCODE'] = test['ID'] + ' X ' + test['product_pred']
  del test['ID'], test['product_pred']
  test.columns = ['Label', 'ID X PCODE']

  submit1 = pd.read_csv('0_fold' + str(r) + '.csv')
  submit2 = pd.read_csv('1_fold' + str(r) + '.csv')
  submit3 = pd.read_csv('2_fold' + str(r) + '.csv')

  test = test.merge(submit1, how='left', on='ID X PCODE')
  test = test.merge(submit2, how='left', on='ID X PCODE')
  test = test.merge(submit3, how='left', on='ID X PCODE')

  test.columns = ['target', 'ID X PCODE', 'Label1', 'Label2', 'Label3']

  fold_train.append(test)

fold_train = pd.concat(fold_train)

In [117]:
test = []
test_columns = test_.columns

for v in test_.values:
  info = v[:8]
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for k in range(len(binary)):
    if k not in index:
      test.append(list(info) + list(binary) + [test_columns[8+k]])
test = pd.DataFrame(test)
test.columns = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
        'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
        '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
        'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'product_pred',]

test = test[['ID', 'product_pred']]
test['ID X PCODE'] = test['ID'] + ' X ' + test['product_pred']
del test['ID'], test['product_pred']

submit1 = pd.read_csv('0_main.csv')
submit2 = pd.read_csv('1_main.csv')
submit3 = pd.read_csv('2_main.csv')

test = test.merge(submit1, how='left', on='ID X PCODE')
test = test.merge(submit2, how='left', on='ID X PCODE')
test = test.merge(submit3, how='left', on='ID X PCODE')

test.columns = ['ID X PCODE', 'Label1', 'Label2', 'Label3']

In [110]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

models = []
models.append(LGBMClassifier(random_state=17, num_trees=300, max_depth = 3))
models.append(LGBMClassifier(random_state=17, num_trees=500, max_depth = 5))
models.append(XGBClassifier(random_state=17, num_trees=300, max_depth = 3))
models.append(XGBClassifier(random_state=17, num_trees=500, max_depth = 5))
models.append(CatBoostClassifier(random_state=17, num_trees=500, max_depth=3))
models.append(CatBoostClassifier(random_state=17, num_trees=1000, max_depth=5))

In [111]:
for i in range(len(models)):
  models[i].fit(fold_train.drop(columns=['target', 'ID X PCODE']), fold_train['target'])

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Learning rate set to 0.415293
0:	learn: 0.1173410	total: 369ms	remaining: 3m 4s
1:	learn: 0.0654411	total: 710ms	remaining: 2m 56s
2:	learn: 0.0395398	total: 1.05s	remaining: 2m 54s
3:	learn: 0.0364164	total: 1.37s	remaining: 2m 50s
4:	learn: 0.0345066	total: 1.71s	remaining: 2m 48s
5:	learn: 0.0343755	total: 2s	remaining: 2m 44s
6:	learn: 0.0335581	total: 2.3s	remaining: 2m 42s
7:	learn: 0.0333065	total: 2.64s	remaining: 2m 42s
8:	learn: 0.0331052	total: 3.03s	remaining: 2m 45s
9:	learn: 0.0327533	total: 3.4s	remaining: 2m 46s
10:	learn: 0.0327113	total: 3.71s	remaining: 2m 45s
11:	learn: 0.0326804	total: 4.03s	remaining: 2m 44s
12:	learn: 0.0326414	total: 4.36s	remaining: 2m 43s
13:	learn: 0.0325655	total: 4.68s	remaining: 2m 42s
14:	learn: 0.0325238	total: 5.01s	remaining: 2m 41s
15:	learn: 0.0325020	total: 5.42s	remaining: 2m 44s
16:	learn: 0.0324869	total: 5.73s	remaining: 2m 42s
17:	learn: 0.0324748	total: 6.04s	remaining: 2m 41s
18:	learn: 0.0324631	total: 6.34s	remaining: 2m 40

In [120]:
predicts = []
for i in range(len(models)):
  predicts.append(models[i].predict_proba(test.drop(columns=['ID X PCODE']))[:, 1])

In [123]:
test['target'] = np.mean(predicts, axis=0)

In [124]:
true_values = []
for v in test_.values:
  binary = v[8:]
  index = [k for k, i in enumerate(binary) if i == 1]
  for k in test_.columns[8:][index]:
    true_values.append(v[0] + ' X ' + k)

In [125]:
test = test[['ID X PCODE', 'target']]
test.columns = ['ID X PCODE', 'Label']

In [126]:
for v in true_values:
  test = test.append({'ID X PCODE':v,'Label':1.0}, ignore_index=True)

In [128]:
test.reset_index(drop=True, inplace=True)
test.to_csv('submission_stack.csv', index=False)

OK